In [42]:
import requests
import sys 
sys.path.append('..')
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pandas as pd
import numpy as np
import pprint


In [43]:
months = {"stycznia": 1,
                "lutego": 2,
                "marca": 3,
                "kwietnia": 4,
                "maja": 5,
                "czerwca": 6,
                "lipca": 7,
                "sierpnia": 8,
                "września": 9,
                "października": 10,
                "listopada": 11,
                "grudnia": 12,
            }

In [77]:
def get_pracuj_job_details(job_element):
    link_element = job_element.select('div[class*="offer__info"]')
    if len(link_element) != 0:
        job_title_element = job_element.find(
            "a", class_="offer-details__title-link"
        )
        job_title = job_title_element.text.strip()

        company_element = job_element.find("p", class_="offer-company")
        company = company_element.text.strip()
        publication_date_element = job_element.find(
            "span", class_="offer-actions__date"
        )
        publication_date_text = publication_date_element.text.strip().split(
            " "
        )[1:]
        month_name = months[publication_date_text[1]]
        # change polish name of month to number ex. sierpnia to 8
        publication_date_text[1] = str(month_name)
        publication_date = "-".join(publication_date_text)
        publication_date = datetime.strptime(
            publication_date, "%d-%m-%Y"
        ).strftime("%Y-%m-%d")

        link_url = job_title_element["href"]
        link_pattern = re.compile(r"https?://([\w.\.\-]+)")
        website_name = link_pattern.match(link_url)[0]
        page_job_element = requests.get(link_url)
        soup_page = BeautifulSoup(page_job_element.content, "html.parser")
        position_element = soup_page.select('li[class*="offer-view"]')
        if "unior" in str(position_element):
            position = "Junior"
        elif "rainee" in str(position_element) or "staż" in str(position_element):
            position = "Trainee"
        else:
            position = "no info"
    else:
        publication_date = np.nan
        company = np.nan
        job_title = np.nan
        position = np.nan
        website_name = np.nan
        link_url = np.nan

    return (publication_date, company, job_title, position, website_name, link_url)


In [74]:
def dict_creator(
    publication_date, company, job_title, position, website_name, link_url
):
    offers_dict = dict()
    offers_dict["publication_date"] = publication_date
    offers_dict["company"] = company
    offers_dict["title"] = job_title
    offers_dict["position"] = position
    offers_dict["website"] = website_name
    offers_dict["link_url"] = link_url

    return offers_dict

In [89]:
def pracuj_page_job_offers(
    url="https://www.pracuj.pl/praca/python;kw/warszawa;wp?rd=30&et=1%2c17&tc=0%2c3%2c2&ws=0&wm=full-office%2chybrid%2chome-office",
) -> list:
    pracuj_list = list()

    flag = True
    try:
        page = requests.get(url)
    except requests.exceptions.ConnectionError as err:
        flag = False
    except requests.exceptions.MissingSchema as err:
        flag = False
    if flag:
        soup = BeautifulSoup(page.content, "html.parser")
        job_elements = soup.select('li[class*="results__list-container-item"]')
        for job_element in job_elements:
            (publication_date, company, job_title, position, website_name, link_url) = get_pracuj_job_details(job_element)
            pracuj_dict = dict_creator(publication_date, company, job_title, position, website_name, link_url)
            pracuj_list.append(pracuj_dict)
    return pracuj_list

In [90]:
pracuj_list = pracuj_page_job_offers("www.wrongpage.pl")
print(pracuj_list)
print(type(pracuj_list))

[]
<class 'list'>


In [81]:
pracuj_list = pracuj_page_job_offers()
df_raw = pd.DataFrame.from_records(pracuj_list)
df = df_raw.copy()
df.drop_duplicates(subset=['company', 'title'], inplace=True, ignore_index=True)
df.dropna(inplace=True)

In [79]:
df

,publication_date,company,title,position,website,link_url
0,2022-09-13,Reckitt Global IT Hub,Junior Demand Analyst - Graduate Program,Junior,https://www.pracuj.pl,https://www.pracuj.pl/praca/junior-demand-anal...
1,2022-09-13,NASK,Inżynier/ka oprogramowania,Junior,https://www.pracuj.pl,https://www.pracuj.pl/praca/inzynier-ka-oprogr...
2,2022-09-13,Leroy Merlin Polska Sp. z o.o.,Stażysta Łańcucha Dostaw,Trainee,https://www.pracuj.pl,https://www.pracuj.pl/praca/stazysta-lancucha-...
3,2022-09-12,THE RABBIT sp. z o.o.,Programista PHP - Magento - Python,Junior,https://www.pracuj.pl,https://www.pracuj.pl/praca/programista-php-ma...
4,2022-09-12,Samsung R&D Institute Poland,Internship - Intern in Broadcast Software Group,Trainee,https://www.pracuj.pl,https://www.pracuj.pl/praca/internship-intern-...
5,2022-09-12,Nokia,Python Developer Working Student,Trainee,https://www.pracuj.pl,https://www.pracuj.pl/praca/python-developer-w...
6,2022-09-12,Nokia,Integration & Testing Working Student,Trainee,https://www.pracuj.pl,https://www.pracuj.pl/praca/integration-testin...
7,2022-09-12,Samsung R&D Institute Poland,DevOps Engineer in Artificial Intelligence Team,Junior,https://www.pracuj.pl,https://www.pracuj.pl/praca/devops-engineer-in...
8,2022-09-12,Nokia,Working Student - Software Test Engineer,Trainee,https://www.pracuj.pl,https://www.pracuj.pl/praca/working-student-so...
9,2022-09-12,Samsung R&D Institute Poland,Internship – C/C++ Intern in Mobile Security Team,Trainee,https://www.pracuj.pl,https://www.pracuj.pl/praca/internship-c-c%2b%...


In [72]:
list(pracuj_list[0].keys())


['publication_date', 'company', 'title', 'position', 'website', 'link_url']